<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/RAG-gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community
!pip install sentence_transformers
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 106.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uni

In [2]:
import os
from pathlib import Path
import subprocess
import shutil
from langchain.text_splitter import TokenTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import pandas as pd
import openai
import faiss
import numpy as np

from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModel, PeftModel
from datasets import load_dataset

import pickle

In [3]:
# 모델 경로
model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model = SentenceTransformer(model_path, device=device)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [4]:
# Load csv file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

dataset = load_dataset("openai/gsm8k", 'socratic')
df = dataset['train'].to_pandas()
df

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/487k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,How many clips did Natalia sell in May? ** Nat...
1,Weng earns $12 an hour for babysitting. Yester...,How much does Weng earn per minute? ** Weng ea...
2,Betty is saving money for a new wallet which c...,How much money does Betty have in the beginnin...
3,"Julie is reading a 120-page book. Yesterday, s...",How many pages did Maila read today? ** Maila ...
4,James writes a 3-page letter to 2 different fr...,How many pages does he write each week? ** He ...
...,...,...
7468,"Very early this morning, Elise left home in a ...",How much did Elise pay in total? ** For the di...
7469,Josh is saving up for a box of cookies. To rai...,How much does Josh make on each bracelet? ** H...
7470,Colin can skip at six times the speed that Bra...,How fast can Tony skip? ** Tony can skip at tw...
7471,"Janet, a third grade teacher, is picking up th...",How many lunches does Janet need? ** Janet nee...


In [6]:
embeddings = []

for i, question in enumerate(df['question'].tolist()):
    embedding = model.encode(question)
    embeddings.append(embedding)

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

# 임베딩을 Numpy 배열로 변환
embedding_matrix = np.array(embeddings).astype('float32')

# 정규화
embedding_matrix_normalized = normalize(embedding_matrix)

# FAISS 인덱스 생성 및 벡터 추가
index = faiss.IndexFlatIP(embedding_matrix_normalized.shape[1])
index.add(embedding_matrix_normalized)

# faiss 인덱스 저장
faiss.write_index(index, 'question_faiss_index-sentence-gsm8k.faiss')

In [7]:
!cp question_faiss_index-sentence-gsm8k.faiss /content/drive/MyDrive/

In [8]:
print(f"Number of questions: {len(df)}")
print(f"Number of embeddings: {len(embeddings)}")

Number of questions: 7473
Number of embeddings: 7473


In [9]:
## 여기서부터는 인덱스 로드해서 사용하는 것. FAISS 인덱스 로드

index = faiss.read_index('question_faiss_index-sentence-gsm8k.faiss')

In [12]:
# 유사도 검색 함수
def search_similar_questions(query_sentence, k=10):
    query_embedding = model.encode([query_sentence])
    query_embedding_normalized = normalize(query_embedding)

    # FAISS 인덱스에서 가장 유사한 벡터 k개 검색
    distances, indices = index.search(np.array([query_embedding_normalized[0]], dtype='float32'), k)

    # 검색된 인덱스와 유사도를 반환
    similar_questions = []
    for i in range(k):
        question = df['question'][indices[0][i]]
        answer = df['answer'][indices[0][i]]
        # plan = df['plan'][indices[0][i]]

        print(f"문서 id: {indices[0][i]}, 유사도: {distances[0][i]}, 질문: {question}")
        similar_questions.append((question, answer))

    return similar_questions

In [14]:
## FAISS 인덱스에서 유사한 벡터 검색
query_sentence = "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?"

similar_questions = search_similar_questions(query_sentence, k=1)
print(similar_questions[0][0], similar_questions[0][1])

문서 id: 51, 유사도: 0.5476208329200745, 질문: Ann is cutting fabric to make curtains. She cuts a 4 foot by 6 foot rectangle for the living room, and a 2 foot by 4 foot rectangle for the bedroom. If the bolt of fabric is 16 feet by 12 feet, how much fabric is left in square feet?
Ann is cutting fabric to make curtains. She cuts a 4 foot by 6 foot rectangle for the living room, and a 2 foot by 4 foot rectangle for the bedroom. If the bolt of fabric is 16 feet by 12 feet, how much fabric is left in square feet? How many square feet was the original bolt of fabric? ** First figure out how many square feet the original bolt of fabric was: 16 feet * 12 feet = <<16*12=192>>192 square feet
How many square feet did Ann take for the living room curtains? ** Then figure out how much fabric Ann took for the living room curtains: 4 feet * 6 feet = <<4*6=24>>24 square feet
How many square feet did Ann take for the bathroom curtains? ** Then figure out how much fabric Ann took for the bathroom curtains: 2 

In [ ]:
# from google.colab import userdata
# from ftplib import FTP

# ftp_server = "121.136.96.223"
# ftp_port = 21
# ftp_username = "donghuna_ftp"
# ftp_password = userdata.get('FTP_PASSWD')
# folder_path = "database/PromptGenerate/"

# ftp = FTP()
# ftp.connect(ftp_server, ftp_port)
# ftp.login(ftp_username, ftp_password)
# ftp.set_pasv(True)

In [ ]:
# # ftp 경로에 파일 저장
# import io

# csv_buffer = io.StringIO()
# df.to_csv(csv_buffer, index=True)

# csv_buffer.seek(0) # buffer의 시작으로 이동
# ftp.storbinary(f"STOR {folder_path}test.csv", io.BytesIO(csv_buffer.getvalue().encode()))